In [1]:
from tool import *

In [2]:
# Quanta가 Commit 한 이후에 GLOP에서 raw data를 다운 받고 해당 다운받은 파일의 경로명을 설정함
frozen = [4]
filepath = 'D:/Downloads/' + [file for file in os.listdir('D:/Downloads/') if 'weeklyThru' in file][-1] # 파일이름에 weeklyThru가 들어가는 파일 중에 가장 최신 파일을 선택함
filepath

'D:/Downloads/weeklyThru_20230418115359.xls'

In [3]:
df = pd.read_excel(filepath) # 해당 엑셀 파일을 데이터프레임으로 불러옴

# 데이터프레임의 컬럼명 중 week 명을 변경해줌
new_columns = []
for col_name in df.columns:
    if re.compile('W\d\d').search(col_name) != None:
        week_name = get_weekname(datetime.date(datetime.date.today().year, int(col_name.strip()[4:6]), int(col_name.strip()[7:9]))) # GLOP에서 다운로드한 엑셀 컬럼명 중에서 주의 명칭을 올해의 LG week로 변경
        new_columns.append(week_name)
    else:
        new_columns.append(col_name)  
df.columns = new_columns

# 해당 주차의 신규 confirm 주차명을 저장함
confirm_week_name = [get_weekname_from(get_weekname(datetime.datetime.strptime(str(df['Plan Name'][0]), '%Y%m%d')), i) for i in frozen]

df = df[df['Category'].isin(['LGE Initial Demand', 'Commit Shipment Plan'])] # category 컬럼에 LG demand와 commit 수량 데이터만 선택
df = df[['Plan Name', 'Model Suffix', 'To Site', 'Category'] + confirm_week_name] # 필요한 최소 컬럼만 추출
df.columns = ['Plan Name', 'Mapping Model.Suffix', 'To Site', 'Category'] + confirm_week_name # 컬럼명을 수정함
df = df.set_index(['Plan Name', 'Mapping Model.Suffix', 'To Site', 'Category']).stack().reset_index() # 확정주차의 수량을 stack화함
df.columns = ['Plan Name', 'Mapping Model.Suffix', 'To Site', 'Category', 'New Confirm Week', 'QTY']
df['Series'] = df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model) # series 모델명을 추가함
df = df.pivot_table('QTY', index=['Plan Name', 'New Confirm Week', 'Series', 'Mapping Model.Suffix', 'To Site'], columns='Category', aggfunc='sum').reset_index()
df = df[df['LGE Initial Demand'] > 0].reset_index(drop=True) # Demand가 0 이상인 것만 선택함
df['DFR'] = round(df['Commit Shipment Plan'] / df['LGE Initial Demand'], 1) * 100 # DRF을 계산

In [4]:
print(f"신규 확정 주차인 {confirm_week_name}의 Demand 합계 : {int(df['LGE Initial Demand'].sum())}")
print(f"신규 확정 주차인 {confirm_week_name}의 Commit 합계 : {int(df['Commit Shipment Plan'].sum())}")

신규 확정 주차인 ['2023-05-15(W20)']의 Demand 합계 : 1860
신규 확정 주차인 ['2023-05-15(W20)']의 Commit 합계 : 1820


In [5]:
df.to_clipboard()

## 1) Quanta의 해당 주차 Commit 내용에 대한 세부 사유가 없을 때

In [5]:
with open('D:/Data/Quanta DFR DB.bin', 'rb') as f: # DFR raw data를 DB에서 불러옴
    dfr = pickle.load(f)

In [6]:
dfr.shape

(336, 9)

In [7]:
dfr = pd.concat([dfr, df]).reset_index(drop=True) # DFR DB에 새로운 df를 추가함

In [8]:
dfr.shape

(359, 9)

In [9]:
# DB에 새롭게 저장함
with open('D:/Data/Quanta DFR DB.bin', 'wb') as f:
    pickle.dump(dfr, f)
with open('D:/Data/DB backup/Quanta DFR DB.bin', 'wb') as f:
    pickle.dump(dfr, f)

## 2) Quanta의 해당 주차 Commit 내용에 대한 세부 사유가 있을 때

In [186]:
df2 = pd.read_clipboard() # 해당 사유 테이블을 복사하여 불러옴, 복사할 때, Plan Name을 추가해줘야 함
df2 = df2.rename(columns={'W13':'LGE Initial Demand'})
df2

,Plan Name,Mapping Model.Suffix,To Site,LGE Initial Demand,Decommit Reason
0,20230320,14T90Q-K.APG5U1,ENUS_V,40,Short plastic shell
1,20230320,14T90R-G.AA56K,EKHQ,30,Short plastic shell
2,20230320,14T90R-G.AA5CK,EKHQ,30,Short plastic shell
3,20230320,14T90R-K.AAB6U1,ENUS_V,10,Short plastic shell
4,20230320,14T90R-K.AAB8U1,ENUS_V,110,Short plastic shell
5,20230320,15U40R-G.AR36K,EKHQ,30,commit
6,20230320,15U40R-G.AR56K,EKHQ,30,commit
7,20230320,15UD40R-G.AX56K,EKHQ,100,commit
8,20230320,15U50P-L.AR2BL,EKHQ,80,commit
9,20230320,15U50Q-G.AP56ML,EKHQ,20,commit


In [7]:
with open('D:/Data/Quanta DFR DB.bin', 'rb') as f:
    dfr = pickle.load(f)

In [191]:
dfr = dfr.merge(df2, how='outer') # 복사한 quanta의 Decommit reason을 DFR raw data DB에 merge 하여 붙여 넣음
dfr

,Plan Name,New Confirm Week,Series,Mapping Model.Suffix,To Site,Commit Shipment Plan,LGE Initial Demand,DFR,Decommit Reason
0,20230320,2023-04-10(W15),14T90Q,14T90Q-K.APG5U1,ENUS_V,0.0,40.0,0.0,Short plastic shell
1,20230320,2023-04-10(W15),14T90R,14T90R-G.AA56K,EKHQ,0.0,30.0,0.0,Short plastic shell
2,20230320,2023-04-10(W15),14T90R,14T90R-G.AA5CK,EKHQ,0.0,30.0,0.0,Short plastic shell
3,20230320,2023-04-10(W15),14T90R,14T90R-K.AAB6U1,ENUS_V,0.0,10.0,0.0,Short plastic shell
4,20230320,2023-04-10(W15),14T90R,14T90R-K.AAB8U1,ENUS_V,0.0,110.0,0.0,Short plastic shell
5,20230320,2023-04-10(W15),15U40R,15U40R-G.AR36K,EKHQ,30.0,30.0,100.0,commit
6,20230320,2023-04-10(W15),15U40R,15U40R-G.AR56K,EKHQ,30.0,30.0,100.0,commit
7,20230320,2023-04-10(W15),15U50P,15U50P-L.AR2BL,EKHQ,80.0,80.0,100.0,commit
8,20230320,2023-04-10(W15),15U50Q,15U50Q-G.AP56ML,EKHQ,20.0,20.0,100.0,commit
9,20230320,2023-04-10(W15),15U50Q,15U50Q-G.AP70ML,EKHQ,70.0,70.0,100.0,commit


In [193]:
# DB 에 업데이트 함
with open('D:/Data/Quanta DFR DB.bin', 'wb') as f:
    pickle.dump(dfr, f)
with open('D:/Data/DB backup/Quanta DFR DB.bin', 'wb') as f:
    pickle.dump(dfr, f)

## DFR DB에서 특정 주차의 DFR 세부 내역을 시리즈별 또는 국가별로 요약할 때

In [101]:
with open('D:/Data/Quanta DFR DB.bin', 'rb') as f:
    df = pickle.load(f)

In [102]:
with open('D:/Data/Quanta DFR DB.bin', 'rb') as f:
    df = pickle.load(f)

# search_plan_name = int(get_weekname(datetime.date.today())[:4] + get_weekname(datetime.date.today())[5:7] + get_weekname(datetime.date.today())[8:10])
search_plan_name = 20230227
df['Quanta Model'] = df['Series'].replace(supplier_model_map)
df = df[df['Plan Name'] == search_plan_name]
# 시리즈별 commit rate 계산
df_series = df.pivot_table(['LGE Initial Demand', 'Commit Shipment Plan'], index=['Series', 'Quanta Model'], aggfunc=sum)
df_series.loc['Sum'] = df_series.sum()
df_series['DFR(%)'] = round(df_series['Commit Shipment Plan'] / df_series['LGE Initial Demand'], 1) * 100
print(f"적용 주차 : {get_weekname(datetime.datetime.strptime(str(search_plan_name), '%Y%m%d'))}")
print(f"신규 확정주차 : {df['New Confirm Week'].values[0]}")
df_series

적용 주차 : 2023-02-27(W09)
신규 확정주차 : 2023-03-20(W12)


,Commit Shipment Plan,LGE Initial Demand,DFR(%)
"(14T90Q, NLJ)",0.0,15.0,0.0
"(14T90R, NLJB)",0.0,255.0,0.0
"(15U40Q, LGH)",29.0,29.0,100.0
"(15U40R, LGHD)",0.0,3000.0,0.0
"(15U50P, LGF)",50.0,50.0,100.0
"(15U50Q, LGHA)",4750.0,4750.0,100.0
"(15U50R, LGHB)",0.0,100.0,0.0
"(16T90Q, NLJA)",0.0,715.0,0.0
"(16T90R, NLJC)",0.0,980.0,0.0
Sum,4829.0,9894.0,50.0


In [240]:
df.to_clipboard() # DFR raw data를 엑셀로 복사

In [236]:
df_series.to_clipboard()

In [103]:
# 국가별 commit rate 계산
df['Country'] =  df['To Site'].replace(site_name_adjust_map).replace(site_map)
df_country = df.pivot_table(['LGE Initial Demand', 'Commit Shipment Plan'], index=['Country', 'To Site'], aggfunc=sum)
df_country.loc['Sum'] = df_country.sum()
df_country['DFR(%)'] = round(df_country['Commit Shipment Plan'] / df_country['LGE Initial Demand'], 1) * 100
print(f"적용 주차 : {get_weekname(datetime.datetime.strptime(str(search_plan_name), '%Y%m%d'))}")
print(f"신규 확정주차 : {df['New Confirm Week'].values[0]}")
df_country

적용 주차 : 2023-02-27(W09)
신규 확정주차 : 2023-03-20(W12)


,Commit Shipment Plan,LGE Initial Demand,DFR(%)
"(Canada, ENCI)",0.0,20.0,0.0
"(Germany, EEDG)",0.0,140.0,0.0
"(Japan, EJJP)",0.0,70.0,0.0
"(Korea, EKHQ)",4829.0,8839.0,50.0
"(Spain, EEES)",0.0,115.0,0.0
"(Taiwan, EATT)",0.0,10.0,0.0
"(UK, EEUK)",0.0,20.0,0.0
"(US, ENUS_V)",0.0,680.0,0.0
Sum,4829.0,9894.0,50.0


In [237]:
df_country.to_clipboard()